# Graphs

## Basics and Terminology

A Graph is a set of vertices `V` and set of edges `E` connecting the vertices. Each vertex has a *degree* which is the number of edges connecting it (and not the number of vertices it connects to).

*Design graphs, not algorithms*. Work on modifying the graph to do what you want it to do, rather than create a new graph algorithm. People have been studying and designing graph algo's for a *really* long time, don't was time trying to create something that probably already exists.

### Directed vs Undirected

An undirected graphs is where the edge (x,y) implies that the edge (y,x) exists
A directed graph is where that implication is not present; i.e, an edge (x,y) does not mean that edge (y,x) exists.

### Weighted vs Unweighted

When there is a cost to a vertex or edge the graph is weighted. Ex: duration; tolls
When there are no costs to vertexes or edges the graph is not weighted

### Simple vs Non-simple

When there are only one edge between vertices, and no self-loops, the graph is said to be *simple*.
When there are multiple edges between vertices, or self-loops, the graph is said to be *non-simple*.

### Sparse vs Dense

When only a small percentage of vertexes actually have edges, the graph is said to be *sparse*.
When many vertexes have edges (let's say O(n^2)) the graph is said to be *dense*.

### Cyclical vs Acyclic

If a cycle exists within the graph the graph is said to be *cyclic*.
If no cycle exists within the graph, the graph is said to be *acyclic*. **Trees** are an acyclic graph.

### Labeled vs Non-Labeled

When each vertex has a unique name the graph is labeled
If each vertex doesn't have a unique name, or just has a logical id, the graph is non-labeled.

If we remove the labels from two different graphs, will they be the same graph? *Polymorphism testing* tests that.

### Path Terminology
Term|Definition
--|--
Path| a path from vertex *a* to *b* is a sequence of edges that can be followed starting at *a* and ending at *b*
Neighbor / Adjacent| two vertices *directly* connected by an edge
Reachable| Vertex *a* is reachable from *b* if a path exists from b to a
Connected| A graph is connected if every vertex is reachable from one another
Cycle| A path that begins and ends at same node

## Implementations

Two major implementations of graphs exist, both with their advantages and disadvantages: Adjacency Matrix and Adjacency Lists.

### Adjacency Matrix

A NxN Matrix (basically 2D Array/List) where each index is a vertex. An edge exists between the two vertexes I and J if [i,j] is 1. An undirected graph would have both [i,j] and [j,i] be equal to 1.

This implementation is better suited for Dense graphs because of the speed of lookup of O(1). Want to see if I and J are connected? Just go to [i,j]!

You may optimize this further fro undirected graphs. Due to the nature of undirected graphs, you're wasting space holding both [i,j] and [j,i] if they mean the same thing.

### Adjacency List

An array of pointers to linked list. Each index in the array of pointers corresponds to a vertex, and each index in the pointed to LL is which vertexes are connected to the vertex.

This implementation is better suited for sparse graphs because the LL will grow linearly with the degree of a vertex (i.e. the more edges, the more LL nodes).

In [54]:
import jdc

In [55]:
class MatrixGraph():
    def __init__(self,matrix):
        self.matrix = matrix
    def has_edge(self,i,j):
        return self.matrix[i][j] != 0
    def get_cost(self,i,j):
        return self.matrix[i][j]
    def make_edge(self,i,j, cost = 1):
        self.matrix[i][j] = cost
    def remove_edge(self, i, j):
        self.make_edge(i, j, 0)

In [67]:
def test_mat_graph():
    mat_graph = MatrixGraph([
        [0,0,1,0],
        [1,1,1,1],
        [0,1,0,1],
        [0,0,0,1],
    ])

    for i in range(0,4):
        text = ""
        for j in range(0,4):
            if mat_graph.has_edge(i,j):
                if text != "":
                    text+= ", "
                text += str(j)
        print(i, "has edges:", text)

    mat_graph.make_edge(3,2,5)
    print("3 has an edge with 2 with cost", mat_graph.get_cost(3, 2))
test_mat_graph()

0 has edges: 2
1 has edges: 0, 1, 2, 3
2 has edges: 1, 3
3 has edges: 3
3 has an edge with 2 with cost 5


### Adjacency List

I'll be using a different implementation of Adjacency Lists then the described one of a Linked List; instead of a LL, I'll be using a dict with the key/value equivalent to `{vertex: weight}`. So each entry in the List will point to a dict instead of a LL. I'm going with this because of the issues with storing a weight with a LL.

I'm keeping track of the basic info: # vertices, and if undirected

First time using defaultdict, so here are some basics

Action|Syntax
--|--
Set Value|ddict[i][j] = val
Check Value|ddict[i].get(j)

In [117]:
from collections import defaultdict
class ListGraph():
    def __init__(self, vertices, is_undirected = True):
        self.vertices = vertices
        self.graph = defaultdict(dict)
        self._isundirected = is_undirected
    
    def add_edge(self, n, m, cost):
        self.graph[n][m] = cost
        if self._isundirected:
            self.graph[m,n] = cost
    
    def get_cost(self, n, m):
        if self.graph[n].get(m):
            return self.graph[n].get(m)
        return -1
    
    def remove_edge(self, n, m):
        self.graph.pop(n,m)


-1


## Traversals

### Breadth First Search

First we go to the root vertex (really just any vertex) and then every vertex that the root has an edge with, and then every vertex that these vertexes have edges with, and so on. We're going to use a Queue data structure. We're adding a parameter of which vertex should be visited first, because graphs aren't too ordered and don't *really* have a root vertex.

Breadth First Search is good for finding the shortest path (in an unweighted or equally-weighted graph)

#### Adjacency List Implementation
We can loop over all the edges of the graph with `for vert in self.graph[vertex]:`. To avoid endless recursion, we're keeping track of which vertexes we've visited.

In [114]:
%%add_to ListGraph
def breadth_first_traversal(self, n):
    visited = [False] * self.vertices
    que = [n]
    visited[n] = True
    while que:
        node = que.pop(0)
        print(node, end=" ")
        
        for i in self.graph[node]:
            if visited[i] is False:
                que.append(i)
                visited[i] = True

In [115]:
def test_bft():
    g = ListGraph(4)
    g.add_edge(0, 1, 3)
    g.add_edge(0, 2, 1)
    g.add_edge(1, 2, 5)
    g.add_edge(1, 0, 3)
    g.add_edge(2, 0, 2)
    g.add_edge(2, 3, 1)
    g.add_edge(3, 3, 4)
    g.breadth_first_traversal(0)
test_bft()

0 1 2 3 

#### Adjacency Matrix Implementation

Exact same logic (oc) but the code is different. We not only loop over all list indices, but check to see if there's an edge between the two vertices.

In [141]:
%%add_to MatrixGraph

def bft(self, vertex):
    visited = [False] * len(self.matrix[0])
    que = [vertex]
    visited[vertex] = True
    while que:
        node = que.pop(0)
        print(node, end=" ")
        for vert in self.matrix[node]:
            print(vert)
            if self.has_edge(node, vert) == False or visited[vert]:
                continue
            visited[vert] = True
            que.append(vert)

In [145]:
def test_matrix_bft():
    mat_graph = MatrixGraph([
        [0,0,1,0],
        [1,1,1,1],
        [0,1,0,1],
        [0,0,0,1],
    ])
    mat_graph.bft(0)
#test_matrix_bft()

UnboundLocalError: local variable 'node' referenced before assignment

### Dijksta's Shortest Path Algorithm

Traversal that takes node weights into account when determining shortest path; similar to Breadth FT but takes node weight/cost into account.